In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import os
import json

# 가격경로 탐색 함수
def find_first(driver, by, locators):
    for loc in locators:
        try:
            return driver.find_element(by, loc)
        except NoSuchElementException:
            continue
    return None

price_xpaths = [
    '//*[@id="content"]/div[2]/div[1]/div[2]/div[4]/div[1]/div[1]/span[1]/em',
    '//*[@id="content"]/div[2]/div[1]/div[2]/div[3]/div[1]/div[1]/span[1]/em',
    '//*[@id="content"]/div[2]/div[1]/div[2]/div[4]/div[2]/div/span/em',
]

# 다운로드 경로
download_dir = "C:/KIMUJUNG/team_project/final_project/crawling/download_json"
driver = webdriver.Chrome()
driver.maximize_window()

# ssg 선물하기
driver.get('https://www.ssg.com/service/gift/main.ssg')
time.sleep(1)

# 유아동 카테고리 클릭
xpath_1 = f'//*[@id="_gift_theme_swiper"]/ul/li[14]/div'
btn_1 = driver.find_element(By.XPATH, xpath_1)
name_txt1 = btn_1.find_element(By.CSS_SELECTOR, 'a > div > img').get_attribute('alt')
# print(name_txt1)
btn_1.click()
time.sleep(2)

for i in range(7, 9):# 8개
    products_data = []

    # 세부 카테고리 클릭
    print(i)
    xpath_2 = f'//*[@id="content"]/div[2]/div[2]/div[2]/label[{i}]'
    btn_2 = driver.find_element(By.XPATH, xpath_2)
    name_txt2 = btn_2.text
    # print(name_txt2)
    btn_2.click()
    time.sleep(2)
    
    output_path = os.path.join(download_dir, "products_babys_" + name_txt2.replace("/", "") +".json")
    
    original_window = driver.current_window_handle
    
    # 상품 선택
    for j in range(1, 101):
        try:
            print(name_txt1, name_txt2, " ", j)
            url = driver.find_element(By.XPATH, f'//*[@id="content"]/div[2]/div[3]/ul/li[{j}]/div/div[1]/div[1]/a')
            driver.execute_script("window.open(arguments[0], '_blank');", url)
            time.sleep(2)
            
            new_handle = [h for h in driver.window_handles if h != original_window][0]
            driver.switch_to.window(new_handle)
            time.sleep(2)

            # 상품명
            name_txt3 = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[2]/h2/span/span').text
            print(name_txt3)

            # 상품 가격
            price_elem = find_first(driver, By.XPATH, price_xpaths)
            price = price_elem.text.replace(",", "").strip()
            print(price)
            
            # 상품 브랜드
            try:
                brand_loc = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/div[2]/h2/span/div/div/a')
                brand = brand_loc.text
                print(brand)
            except NoSuchElementException:
                print('브랜드 없음')
                brand = None

            # 상품 이미지
            try:
                img_elem = driver.find_element(By.XPATH, '//*[@id="mainImg"]')
                link_save = img_elem.get_attribute('src')
                print(link_save)
            except NoSuchElementException:
                pass
                
            # 상품 옵션
            option_list = []
            try:
                # 옵션창 열기
                option_open = driver.find_element(By.XPATH, '//*[@id="_ordOpt_area"]/dl/dd/div')
                option_open.click()
                option_ul = driver.find_element(By.XPATH, '//*[@id="_ordOpt_area"]/dl/dd/div/div/ul')
                li_elements = option_ul.find_elements(By.TAG_NAME, "li")
                for k in li_elements:
                    option_text = k.find_element(By.TAG_NAME, "a").text
                    option_list.append(option_text)

            except NoSuchElementException:
                print('옵션 없음')
                pass
            print(option_list)

            # 상품 링크
            current_url = driver.current_url
            print("상품 링크:", driver.current_url)

            # 리뷰내용
            review_list = []
            try:
                xpath_3 = f'//*[@id="_cdtl_dtlcont_wrap"]/div[1]/div/div[1]/ul/li[3]/a'
                btn_3 = driver.find_element(By.XPATH, xpath_3)
                btn_3.click()
                time.sleep(2)

                for l in range(1, 101):
                    try:
                        review_num = ((l - 1) % 10) + 1
                        xpath_4 = f'//*[@id="item_rvw_list"]/li[{review_num}]/div[1]/a/div/div[3]/p'
                        review = driver.find_element(By.XPATH, xpath_4)
                        review_text = review.text
                        review_list.append(review_text)
                        # 다음페이지 클릭
                        if l % 10 == 0:
                            try:
                                page_num = l // 10
                                btn_4 = driver.find_element(By.XPATH, f'//*[@id="comment_navi_area"]/div/a[{page_num}]')
                                btn_4.click()
                                time.sleep(2)

                            except NoSuchElementException:
                                pass
                    except NoSuchElementException:
                        pass
            except NoSuchElementException:
                pass

            print(review_list)

            products_data.append({
                "brand_name": brand,
                "product_name": name_txt3,
                "category_main": name_txt1,
                "category_sub": name_txt2,
                "options": option_list,
                "product_url": current_url,
                "thumbnail_url": link_save,
                "review_content": review_list,
                "price": price
            })

            driver.close()
            driver.switch_to.window(original_window)
            time.sleep(2)
        except NoSuchElementException:
            pass
        
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(products_data, f, ensure_ascii=False, indent=2)

driver.quit()

7
유아동 학습/교구   1
스토케 뮤테이블 세트 테이블+의자 2 (색상선택)
702000
스토케
https://sitem.ssgcdn.com/58/40/64/item/1000573644058_i1_1200.jpg
['화이트', '라일락', '스톰그레이', '클로버그린']
상품 링크: https://www.ssg.com/item/itemView.ssg?itemId=1000573644058&siteNo=6009&salestrNo=1004
[]
유아동 학습/교구   2
스토케 뮤테이블 플레이 테이블 (색상선택)
436000
스토케
https://sitem.ssgcdn.com/79/40/64/item/1000573644079_i1_1200.jpg
['화이트', '라일락', '스톰그레이', '클로버그린']
상품 링크: https://www.ssg.com/item/itemView.ssg?itemId=1000573644079&siteNo=6009&salestrNo=1004
['진작 일찍들였어도 좋을꺼같아요 진짜튼튼합니다 활용도 최고에요', '판을 갈 수 있다는게 매력적인 테이블 입니다 만족하며 사용중입니다', '진작 일찍들였어도 좋을꺼같아요 진짜튼튼합니다 활용도 최고에요', '판을 갈 수 있다는게 매력적인 테이블 입니다 만족하며 사용중입니다', '진작 일찍들였어도 좋을꺼같아요 진짜튼튼합니다 활용도 최고에요', '판을 갈 수 있다는게 매력적인 테이블 입니다 만족하며 사용중입니다', '진작 일찍들였어도 좋을꺼같아요 진짜튼튼합니다 활용도 최고에요', '판을 갈 수 있다는게 매력적인 테이블 입니다 만족하며 사용중입니다', '진작 일찍들였어도 좋을꺼같아요 진짜튼튼합니다 활용도 최고에요', '판을 갈 수 있다는게 매력적인 테이블 입니다 만족하며 사용중입니다', '진작 일찍들였어도 좋을꺼같아요 진짜튼튼합니다 활용도 최고에요', '판을 갈 수 있다는게 매력적인 테이블 입니다 만족하며 사용중입니다', '진작 일찍들였어도 좋을꺼같아요 진짜튼튼합니다 활용도 최고에요', '판을 갈 수 있다는

In [29]:
len(products_data[0]['review_content'])

10